# Introduction to Seq2Seq Networks using Pytorch

https://github.com/bentrevett/pytorch-seq2seq/blob/master/3%20-%20Neural%20Machine%20Translation%20by%20Jointly%20Learning%20to%20Align%20and%20Translate.ipynb

Nesse notebook irei realizar um pretreino da rede com atenção para traduzir frases de inglês -> inglês e depois aplicar o encoder aprendido na rede de pontuação.

Compararei os resultados da função de perda e perplexidade para verificar se houve alguma melhoria nos resultados por conta desse pre-treino.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import TranslationDataset, Multi30k, IWSLT
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

import string

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
!python -m spacy download en
!python -m spacy download de

You should consider upgrading via the 'pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
You should consider upgrading via the 'pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [4]:
spacy_en = spacy.load('en')

In [1]:
def tokenize_en_punc(text):
    r = list()
    for tok in spacy_en.tokenizer(text):
        if tok.text in string.punctuation:
            try:
                r.pop()
            except:
                pass
            r.append(tok.text)
        else:
            r.append(' ')
    return r

def tokenize_en(text):
    r = list()
    for tok in spacy_en.tokenizer(text):
        if tok.text in string.punctuation:
            continue
        else:
            r.append(tok.text)
    return r

In [6]:
SRC = Field(tokenize=tokenize_en,
            init_token='<sos>',
            eos_token='<eos>',
            lower=True)

TRG = Field(tokenize=tokenize_en_punc,
            init_token='<sos>',
            eos_token='<eos>',
            lower=True)

In [2]:
train_data, valid_data, test_data = Multi30k.splits(exts=('.en', '.en'),
                                                    fields=(SRC, TRG))

auto_train_data, auto_valid_data, auto_test_data = Multi30k.splits(exts=('.en', '.en'),
                                                    fields=(SRC, SRC))

NameError: name 'Multi30k' is not defined

In [8]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


In [9]:
def show_complete_example(example):
    sentence = ''
    for src, trg in zip(example['src'], example['trg']):
        sentence += src
        if trg != ' ':
            trg += ' '
        sentence += trg
    return sentence

print(vars(train_data.examples[0]))

show_complete_example(vars(train_data.examples[0]))

{'src': ['two', 'young', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes'], 'trg': [' ', ',', ' ', ' ', ' ', ' ', ' ', ' ', '.']}


'two young, white males are outside near many bushes. '

In [3]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

NameError: name 'SRC' is not defined

In [4]:
TRG.vocab.stoi

NameError: name 'TRG' is not defined

In [12]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 5879
Unique tokens in target (en) vocabulary: 19


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
batch_size = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=batch_size,
    device=device)

auto_train_iterator, auto_valid_iterator, auto_test_iterator = BucketIterator.splits(
    (auto_train_data, auto_valid_data, auto_test_data),
    batch_size=batch_size,
    device=device)

In [15]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional=True)
        self.dense = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.rnn(embedded)
        hidden = torch.tanh(self.dense(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)))
        return outputs, hidden

In [16]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias=False)
        
    def forward(self, hidden, encoder_outputs):
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        
        attention = self.v(energy).squeeze(2)
        
        return F.softmax(attention, dim=1)

In [17]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

In [18]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [19]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5


attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [20]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5879, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (dense): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(19, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=19, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [21]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 7,977,235 trainable parameters


In [22]:
optimizer = optim.Adam(model.parameters())
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [23]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [24]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            
            src = batch.src
            trg = batch.trg
            
            output = model(src, trg, 0)
            
            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            
            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
            
    return epoch_loss / len(iterator)

In [25]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### Rede sem pre-treino:

In [26]:
filename = '4.1-punctuation-no-pretraining.pt'
N_EPOCHS = 20
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '../models/' + filename)
        
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 38s
	Train Loss: 0.529 | Train PPL:   1.697
	 Val. Loss: 1.151 |  Val. PPL:   3.160
Epoch: 02 | Time: 0m 39s
	Train Loss: 0.371 | Train PPL:   1.450
	 Val. Loss: 0.992 |  Val. PPL:   2.697
Epoch: 03 | Time: 0m 39s
	Train Loss: 0.334 | Train PPL:   1.397
	 Val. Loss: 0.858 |  Val. PPL:   2.359
Epoch: 04 | Time: 0m 39s
	Train Loss: 0.324 | Train PPL:   1.383
	 Val. Loss: 0.705 |  Val. PPL:   2.024
Epoch: 05 | Time: 0m 39s
	Train Loss: 0.321 | Train PPL:   1.379
	 Val. Loss: 0.714 |  Val. PPL:   2.043
Epoch: 06 | Time: 0m 39s
	Train Loss: 0.243 | Train PPL:   1.275
	 Val. Loss: 0.818 |  Val. PPL:   2.266
Epoch: 07 | Time: 0m 39s
	Train Loss: 0.222 | Train PPL:   1.248
	 Val. Loss: 0.318 |  Val. PPL:   1.375
Epoch: 08 | Time: 0m 39s
	Train Loss: 0.171 | Train PPL:   1.186
	 Val. Loss: 0.580 |  Val. PPL:   1.786
Epoch: 09 | Time: 0m 39s
	Train Loss: 0.187 | Train PPL:   1.205
	 Val. Loss: 0.982 |  Val. PPL:   2.670
Epoch: 10 | Time: 0m 39s
	Train Loss: 0.119 | Train PPL

In [27]:
model.load_state_dict(torch.load('../models/' + filename))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 0.220 | Test PPL:   1.246 |


In [28]:
def tokens_to_sentence(tokens, vocab):
    words = list()
    for tok in tokens:
        words.append(vocab.itos[tok])
    return ' '.join(words)

test_batch = next(iter(test_iterator))
src = test_batch.src
trg = test_batch.trg

model.eval()
output = torch.argmax(model.forward(src, trg, 0), -1)


for i, (src, out, tar) in enumerate(zip(src.permute(1, 0), output.permute(1, 0), trg.permute(1, 0))):
    print("Sentence ", i)
    print("Original Sentence:", tokens_to_sentence(src, SRC.vocab))

Sentence  0
Original Sentence: <sos> a <unk> brown dog walking on the grass <eos>
Sentence  1
Original Sentence: <sos> two men sit talking near a stone building <eos>
Sentence  2
Original Sentence: <sos> the two kids are playing at the playground <eos>
Sentence  3
Original Sentence: <sos> the boy is outside enjoying a summer day <eos>
Sentence  4
Original Sentence: <sos> girl wearing radio t shirt has open mouth <eos>
Sentence  5
Original Sentence: <sos> two men breakdancing with a crowd looking on <eos>
Sentence  6
Original Sentence: <sos> little girl kicks black object at karate class <eos>
Sentence  7
Original Sentence: <sos> a kid crosscountry skis wearing the number <unk> <eos>
Sentence  8
Original Sentence: <sos> a man with graying hair <unk> his beard <eos>
Sentence  9
Original Sentence: <sos> white duck <unk> its wings in the water <eos>
Sentence  10
Original Sentence: <sos> the teen jumps the hill with his bicycle <eos>
Sentence  11
Original Sentence: <sos> the boy jumps onto 

### Rede com pre-treino

In [29]:
attn = None
enc = None
dec = None

model = None

In [30]:
INPUT_DIM = len(SRC.vocab)
#OUTPUT_DIM = len(TRG.vocab)
OUTPUT_DIM = INPUT_DIM
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5


attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [31]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5879, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (dense): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5879, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=5879, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [32]:
optimizer = optim.Adam(model.parameters())
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=SRC_PAD_IDX)

In [33]:
filename = '4.1-punctuation-with-pretraining.pt'
N_EPOCHS = 20
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, auto_train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, auto_valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '../models/' + filename)
        
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 1m 12s
	Train Loss: 5.034 | Train PPL: 153.513
	 Val. Loss: 4.781 |  Val. PPL: 119.236
Epoch: 02 | Time: 1m 12s
	Train Loss: 3.084 | Train PPL:  21.854
	 Val. Loss: 1.775 |  Val. PPL:   5.903
Epoch: 03 | Time: 1m 11s
	Train Loss: 1.154 | Train PPL:   3.172
	 Val. Loss: 0.813 |  Val. PPL:   2.254
Epoch: 04 | Time: 1m 12s
	Train Loss: 0.545 | Train PPL:   1.725
	 Val. Loss: 0.436 |  Val. PPL:   1.547
Epoch: 05 | Time: 1m 11s
	Train Loss: 0.302 | Train PPL:   1.352
	 Val. Loss: 0.281 |  Val. PPL:   1.324
Epoch: 06 | Time: 1m 12s
	Train Loss: 0.182 | Train PPL:   1.199
	 Val. Loss: 0.205 |  Val. PPL:   1.227
Epoch: 07 | Time: 1m 11s
	Train Loss: 0.107 | Train PPL:   1.113
	 Val. Loss: 0.196 |  Val. PPL:   1.217
Epoch: 08 | Time: 1m 11s
	Train Loss: 0.080 | Train PPL:   1.083
	 Val. Loss: 0.260 |  Val. PPL:   1.297
Epoch: 09 | Time: 1m 12s
	Train Loss: 0.066 | Train PPL:   1.069
	 Val. Loss: 0.189 |  Val. PPL:   1.207
Epoch: 10 | Time: 1m 11s
	Train Loss: 0.064 | Train PPL

In [34]:
attn = None
dec = None

model = None

In [35]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5


attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
#enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [36]:
def init_weights(m):
    for name, param in m.named_parameters():
        if not name.startswith('encoder'):
            nn.init.normal_(param.data, mean=0, std=0.01)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5879, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (dense): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(19, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=19, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [37]:
optimizer = optim.Adam(model.parameters())
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [38]:
filename = '4.1-punctuation-with-pretraining.pt'
N_EPOCHS = 20
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '../models/' + filename)
        
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 39s
	Train Loss: 0.481 | Train PPL:   1.618
	 Val. Loss: 0.824 |  Val. PPL:   2.280
Epoch: 02 | Time: 0m 39s
	Train Loss: 0.385 | Train PPL:   1.469
	 Val. Loss: 0.731 |  Val. PPL:   2.076
Epoch: 03 | Time: 0m 39s
	Train Loss: 0.353 | Train PPL:   1.424
	 Val. Loss: 0.842 |  Val. PPL:   2.320
Epoch: 04 | Time: 0m 39s
	Train Loss: 0.317 | Train PPL:   1.373
	 Val. Loss: 0.403 |  Val. PPL:   1.497
Epoch: 05 | Time: 0m 39s
	Train Loss: 0.302 | Train PPL:   1.353
	 Val. Loss: 0.651 |  Val. PPL:   1.917
Epoch: 06 | Time: 0m 39s
	Train Loss: 0.213 | Train PPL:   1.237
	 Val. Loss: 0.734 |  Val. PPL:   2.084
Epoch: 07 | Time: 0m 39s
	Train Loss: 0.217 | Train PPL:   1.242
	 Val. Loss: 0.566 |  Val. PPL:   1.761
Epoch: 08 | Time: 0m 39s
	Train Loss: 0.161 | Train PPL:   1.175
	 Val. Loss: 0.235 |  Val. PPL:   1.264
Epoch: 09 | Time: 0m 39s
	Train Loss: 0.144 | Train PPL:   1.154
	 Val. Loss: 0.147 |  Val. PPL:   1.158
Epoch: 10 | Time: 0m 39s
	Train Loss: 0.116 | Train PPL

In [39]:
model.load_state_dict(torch.load('../models/' + filename))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 0.099 | Test PPL:   1.104 |


In [40]:
def tokens_to_sentence(tokens, vocab):
    words = list()
    for tok in tokens:
        words.append(vocab.itos[tok])
    return ' '.join(words)

test_batch = next(iter(test_iterator))
src = test_batch.src
trg = test_batch.trg

model.eval()
output = torch.argmax(model.forward(src, trg, 0), -1)


for i, (src, out, tar) in enumerate(zip(src.permute(1, 0), output.permute(1, 0), trg.permute(1, 0))):
    print("Sentence ", i)
    print("Original Sentence:", tokens_to_sentence(src, SRC.vocab))

Sentence  0
Original Sentence: <sos> a <unk> brown dog walking on the grass <eos>
Sentence  1
Original Sentence: <sos> two men sit talking near a stone building <eos>
Sentence  2
Original Sentence: <sos> the two kids are playing at the playground <eos>
Sentence  3
Original Sentence: <sos> the boy is outside enjoying a summer day <eos>
Sentence  4
Original Sentence: <sos> girl wearing radio t shirt has open mouth <eos>
Sentence  5
Original Sentence: <sos> two men breakdancing with a crowd looking on <eos>
Sentence  6
Original Sentence: <sos> little girl kicks black object at karate class <eos>
Sentence  7
Original Sentence: <sos> a kid crosscountry skis wearing the number <unk> <eos>
Sentence  8
Original Sentence: <sos> a man with graying hair <unk> his beard <eos>
Sentence  9
Original Sentence: <sos> white duck <unk> its wings in the water <eos>
Sentence  10
Original Sentence: <sos> the teen jumps the hill with his bicycle <eos>
Sentence  11
Original Sentence: <sos> the boy jumps onto 